In [43]:
%%writefile msgs_dump.py
from telethon.sync import TelegramClient
from telethon import connection
import configparser
import json
import pandas as pd

from telethon.sync import TelegramClient
from telethon import connection

# для корректного переноса времени сообщений в json
from datetime import date, datetime

# классы для работы с каналами
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch

# класс для работы с сообщениями
from telethon.tl.functions.messages import GetHistoryRequest

async def dump_all_messages(channel):
	offset_msg = 0    # номер записи, с которой начинается считывание
	limit_msg = 100   # максимальное число записей, передаваемых за один раз

	all_messages = []   # список всех сообщений
	total_messages = 0
	total_count_limit = 0  # поменяйте это значение, если вам нужны не все сообщения

	class DateTimeEncoder(json.JSONEncoder):
		'''Класс для сериализации записи дат в JSON'''
		def default(self, o):
			if isinstance(o, datetime):
				return o.isoformat()
			if isinstance(o, bytes):
				return list(o)
			return json.JSONEncoder.default(self, o)

	while True:
		history = await client(GetHistoryRequest(
			peer=channel,
			offset_id=offset_msg,
			offset_date=None, add_offset=0,
			limit=limit_msg, max_id=0, min_id=0,
			hash=0))
		if not history.messages:
			break
		messages = history.messages
		for message in messages:
			all_messages.append(message.to_dict())
		offset_msg = messages[len(messages) - 1].id
		total_messages = len(all_messages)
		if total_count_limit != 0 and total_messages >= total_count_limit:
			break

	with open('channel_messages.json', 'w', encoding='utf8') as outfile:
		 json.dump(all_messages, outfile, ensure_ascii=False, cls=DateTimeEncoder)

async def main():
	url = input("Введите ссылку на канал или чат: ")
	channel = await client.get_entity(url)
	await dump_all_messages(channel)

config = configparser.ConfigParser()
config.read("config.ini")

# Присваиваем значения внутренним переменным
api_id   = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']
username = config['Telegram']['username']

client = TelegramClient(username, api_id, api_hash)
client.start()

async def main():
# 	url = input("Введите ссылку на канал или чат: ")
# 	channel = await client.get_entity(url)
# 	await dump_all_messages(channel)
    participants_info = []
    dialogs = await client.get_dialogs()
    for dialog in dialogs:
        if dialog.name == 'ШАД 16+':
            participants = await client.get_participants(dialog.input_entity)
            for part in participants:
                participants_info.append(
                    (
                        f'{part.first_name} {part.last_name}',
                        part.username,
                        str(part.phone),
                    )
                )
    pd.DataFrame(participants_info, columns=['name', 'username', 'phone']).to_csv('shad.csv', index=False)

    
    
with client:
	client.loop.run_until_complete(main())

Overwriting msgs_dump.py


In [47]:
import pandas as pd
pd.read_csv('/Users/blackjacker/code/unipopcorn/unipop_cornbot/shad.csv')[10:]

,name,username,phone
10,Elijah Kamski,DocSherLOCK,None
11,Anna Tolstikova,annatolstikova,None
12,Maksim K 🪶,mak_kol,None
13,Ilya Blinnikov,i_blinnikov,None
14,Norbert Nemesh,norberrt,None
...,...,...,...
942,Dmitry Popov,foksly,None
943,Natalia None,natashadcan,None
944,Vladimir I,iofik,None
945,Pavel Sushin,psushin,None
